In [1]:
import pandas as pd

# IMPORTIAMO LA TABELLA DI EXCEL NEL NOSTRO NOTEBOOK: (useremo la libreria "pandas")

#stampare i dati
pd.read_excel("file_di_dati.xls", sheet_name="porzioni di viaggio") 
pd.read_excel("file_di_dati.xls", sheet_name="tempi di percorrenza a vuoto") 

#salvare i dati dentro una variabile
xlsx1 = pd.read_excel("file_di_dati.xls", sheet_name="porzioni di viaggio") 
xlsx2 = pd.read_excel("file_di_dati.xls", sheet_name="tempi di percorrenza a vuoto") 

#filtrare e salvare solo alcune colonne dentro una variabile
SP_i = pd.read_excel("file_di_dati.xls",'porzioni di viaggio', usecols='C')          #colonna Stazioni di Partenza
SP_i=SP_i.values.tolist();
SP_i.pop(0);
SP_i.pop();

SA_i = pd.read_excel("file_di_dati.xls",'porzioni di viaggio', usecols='D')          #colonna Stazioni di Arrivo
SA_i=SA_i.values.tolist();
SA_i.pop(0);
SA_i.pop();

S_i = pd.read_excel("file_di_dati.xls",'porzioni di viaggio', usecols='E')          #colonna Tempi di Inizio
S_i=S_i.values.tolist();
S_i.pop(0);
S_i.pop();

A_i = pd.read_excel("file_di_dati.xls",'porzioni di viaggio', usecols='F')           #colonna Tempi di Fine
A_i=A_i.values.tolist();
A_i.pop(0);
A_i.pop();

P_extra = pd.read_excel("file_di_dati.xls",'porzioni di viaggio', usecols='B')        #colonna Passeggeri Extra
P_extra = P_extra.values.tolist();
P_extra.pop(0);
P_extra.pop();

# STAMPE PER VERIFICARE L'IMPORTAZIONE CORRETTA DEI DATI

#xlsx1 #(levo l'hashtag se voglio vedere gli elementi nella variabile)
#xlsx2 #(levo l'hashtag se voglio vedere gli elementi nella variabile)
#S_i #(levo l'hashtag se voglio vedere gli elementi nella variabile)
#A_i #(levo l'hashtag se voglio vedere gli elementi nella variabile)
#SP_i
#SA_i
#P_extra

FileNotFoundError: [Errno 2] No such file or directory: 'file_di_dati.xls'

In [ ]:
import time
stazioni=(xlsx2.values.tolist())[0]            # prendiamo dalla xlsx2 (relativa al foglio "tempi percorrenza a vuoto" )
stazioni.pop(0);                               # la prima colonna ed eliminiamo tramite "pop" la prima riga
for i in stazioni:
    i.rstrip();

DizionarioStazioni = {stazioni[i]:i for i in range(16)}

p=len(SP_i);

for i in range(p):
    stringa=SP_i[i][0];
    stringa.rstrip(); 
    SP_i[i][0]=DizionarioStazioni[stringa]
    stringa=SA_i[i][0];
    stringa.rstrip();
    SA_i[i][0]=DizionarioStazioni[stringa]
    
    
# RIEMPIAMO LA MATRICE DELLE TRATTE (ogni elemento = Partenza --> Arrivo)

tempi_vuoto = xlsx2.values.tolist(); # trasformo tabella in matrice

tempi_vuoto.pop(0);      # elimino la prima riga della matrice (nomi fermate)
for i in range(16):
    tempi_vuoto[i].pop(0);       # elimino la prima colonna di ogni riga della matrice (nomi fermate)
for i in range(16):
    tempi_vuoto[i][i]=0;      # la diagonale principale ha tempi di percorrenza a vuoto NULLI (rimango nella stessa fermata) 
for i in range(16):
    for j in range(16):
        tempi_vuoto[i][j]=tempi_vuoto[j][i]; #HP: I tempi di percorrenza a vuoto sono uguali in tutti e 2 i versi di percorrenza 


In [3]:
import mip
import datetime
import networkx as nx


# >>> FORMULAZIONE >>>

# Richiesta del numero di Autobus Aggiuntivi da usare:

k = int(input('Inserire il Numero di Autobus aggiuntivi: '))
print ("\n--> Numero Autobus aggiuntivi inseriti: ", k)

m=[]
P=[]
P_recuperati = []
tot_passeggeri = 0    # Definiamo una variabile pari a 0 dei Passeggeri Totali che verrà mano a mano aggiornata al variare di k

for i in range(p):
    P.append(P_extra[i][0])
P.insert(0,0)
P.append(0)


# NODI:

Di = 0          # NODO DI INIZIO --> I nodi delle tratte cominciano dal numero "1"
Df = p+2          # NODO DI FINE --> I nodi delle tratte finiscono al numero "156"

V=[Di]
for x in range(p):
    V.append(x+1)
V.append(Df)


# ARCHI:

A=[]
for i in range(p):
    for j in range(p):
        if i != j and A_i[i][0].timestamp() + datetime.timedelta(minutes = tempi_vuoto[SA_i[i][0]][SP_i[j][0]]).seconds <= S_i[j][0].timestamp():
            A.append((i+1,j+1))          # Tramutiamo la Data + Ora del foglio excel in un equivalente temporale tramite
                                         # il comando timestamp (numero di secondi dalla data del 1 Gennaio 1970 da definizione)
for i in range(p+1):
    A.append((0, i+1))
for i in range(p+1):    
    A.append((i+1, p+2))
A.append((0,p+2)) 

for b in range(k):
    X=[]
    m.append(mip.Model())
    
    X={(i,j): m[b].add_var(var_type = mip.BINARY) for (i,j) in A}     # Variabile Booleana (se prendo il relativo arco (i,j))
 

# VINCOLI:

    for i in V:
        if i==0:
            m[b].add_constr(mip.xsum(X[i,j] for j in V if (i,j) in A) -      # Outgoing flow from i
                         mip.xsum(X[j,i] for j in V if (j,i) in A) ==        # Incoming flow into i
                         1) #1= num bus
        if i==p+2:
            m[b].add_constr(mip.xsum(X[i,j] for j in V if (i,j) in A) -      # Outgoing flow from i
                         mip.xsum(X[j,i] for j in V if (j,i) in A) ==        # Incoming flow into i
                         -1)
        if i!=0 and i!=p+2:
            m[b].add_constr(mip.xsum(X[i,j] for j in V if (i,j) in A) -      # Outgoing flow from i
                         mip.xsum(X[j,i] for j in V if (j,i) in A) ==        # Incoming flow into i
                         0)

            
# FUNZIONE OBIETTIVO:   Massimizzo la somma delle persone extra se l'arco viene scelto (e quindi se X[i,j] == 1)

    m[b].objective = mip.maximize(mip.xsum(P[i] * X[i,j] for (i,j) in A))    

    
# STAMPA della SOLUZIONE:

    status = m[b].optimize()  
    
    P_recuperati.append(m[b].objective_value)
    
    print(f"\nIl Bus {b} raccoglie {P_recuperati[b]} passeggeri: \n")
    eliminare = []
    for (i,j) in A:
        if(X[i,j].x):
            print(f"prendo l'arco {i} --> {j}")
            if j == p+2:
                eliminare.append((i,j))
            else:
                for z in V:
                    if (z,j) in A:
                        eliminare.append((z,j))
            
    for x in eliminare:        # Ad ogni bus che "spediamo" sulla rete, togliamo i relativi archi che vengono scelti
        A.remove(x)            # Così mano a mano avremo un grafo con meno archi (e quindi meno persone extra da raccogliere)
        
    tot_passeggeri += m[b].objective_value
    print(f"\n--> Il Totale dei passeggeri recuperati fino ad ora è: {tot_passeggeri} \n \n")   


        



--> Numero Autobus aggiuntivi inseriti:  30


NameError: name 'p' is not defined

In [24]:
print(A)

[(1, 4), (1, 6), (1, 7), (1, 9), (1, 13), (1, 28), (1, 29), (1, 34), (1, 47), (1, 51), (1, 52), (1, 53), (1, 54), (1, 55), (1, 57), (1, 58), (1, 59), (1, 60), (1, 61), (1, 62), (1, 64), (1, 65), (1, 66), (1, 67), (1, 68), (1, 69), (1, 70), (1, 72), (1, 73), (1, 74), (1, 75), (1, 76), (1, 77), (1, 78), (1, 80), (1, 81), (1, 82), (1, 83), (1, 84), (1, 85), (1, 103), (1, 104), (1, 105), (1, 106), (1, 108), (1, 109), (1, 110), (1, 111), (1, 112), (1, 113), (1, 114), (1, 116), (1, 119), (1, 120), (1, 123), (1, 124), (1, 125), (1, 126), (1, 127), (1, 129), (1, 130), (1, 131), (1, 132), (1, 133), (1, 134), (1, 150), (2, 9), (2, 13), (2, 28), (2, 29), (2, 34), (2, 47), (2, 51), (2, 52), (2, 53), (2, 54), (2, 55), (2, 57), (2, 58), (2, 59), (2, 60), (2, 61), (2, 62), (2, 64), (2, 65), (2, 66), (2, 67), (2, 68), (2, 69), (2, 70), (2, 72), (2, 73), (2, 74), (2, 75), (2, 76), (2, 77), (2, 78), (2, 80), (2, 81), (2, 82), (2, 83), (2, 84), (2, 85), (2, 103), (2, 104), (2, 105), (2, 106), (2, 108), (

In [ ]:
FINE CODICE